In [1]:
import sys, os, fnmatch, datetime, subprocess, imp
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from maxentpy import maxent_fast as ment
from maxentpy.maxent_fast import load_matrix

In [2]:
import math

In [3]:
match_dct = {
    'A':'T',
    'C':'G',
    'T':'A',
    'G':'C',
}

In [5]:
def log_to_prob(log):
    return math.exp(log)/(1 + math.exp(log))

In [6]:
def reverse_complement(umi):
    if len(umi) == 0:
        return ''
    try:
        this = match_dct[umi[-1:]]
    except:
        print(umi)
        this = ''
    return this + reverse_complement(umi[:-1])

In [7]:
matrix3 = load_matrix(3)

In [8]:
complete_df = pd.read_csv('new_maxent_mers.csv')
complete_df = complete_df.drop(['insert_length', 'deletion_length', 'in_genotype', 'target', 'lib'], axis=1)
complete_df = complete_df.drop(['insert_length', 'deletion_length', 'in_genotype', 'target', 'lib'], axis=1)
complete_df['maxent_score'] = complete_df.maxent_mer.apply(lambda x: ment.score3(x, matrix=matrix3))
complete_df['maxent_prob'] = complete_df.maxent_score.apply(lambda x: log_to_prob(x))

/home/clara/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
wild_type = pd.read_csv('061318_exonskipping_library.csv')

In [ ]:
wild_type['sequence_23'] = wild_type['Designed 61-bp target site (37i-24e, AG)'].apply(lambda x: x[17:40])

In [ ]:
wild_type['maxent_mer'] = wild_type['Designed 61-bp target site (37i-24e, AG)'].apply(lambda x: ment.score3(x[17:40], matrix = matrix3))

In [ ]:
wild_type = wild_type.rename(columns={'Identifier number':'id'})

In [ ]:
wild_type_dict = pd.Series(wild_type.maxent_mer.values, index=wild_type.id).to_dict()

In [ ]:
plot = plt.hist(wild_type.maxent_mer, bins=50)
txt = plt.title('Wild type maxent score distribution')

In [ ]:
plot = plt.hist(df.score, bins=50)
txt = plt.title('Predicted maxent score distribution')

In [ ]:
insertion = complete_df.loc[complete_df.Category == 'ins']
ins_percentage = insertion.groupby(['library_key'])['Predicted frequency'].sum().reset_index('library_key')
ins_percentage_dict = pd.Series(ins_percentage['Predicted frequency'].values, index = ins_percentage.library_key).to_dict()

In [ ]:
insertion['Predicted frequency'] = insertion.apply(lambda x: x['Predicted frequency']/ins_percentage_dict[x.library_key], axis=1)

In [ ]:
insertion['score'] = insertion.maxent_score * insertion['Predicted frequency']

In [ ]:
ins_weighted = insertion.groupby('library_key')['score'].sum()
ins_weighted = ins_weighted.reset_index('library_key').rename(columns={'library_key':'id'})
plot = plt.hist(ins_weighted.score, bins=50)

In [ ]:
ins_weighted['wild_type'] = ins_weighted.id.apply(lambda x: wild_type_dict[x])

In [ ]:
plot = plt.scatter(ins_weighted.score, ins_weighted.wild_type)
plt.xlabel('insertions')
plt.ylabel('wild type')
plt.title('Insertion vs wildtype score distribution')
round(np.corrcoef(ins_weighted.score, ins_weighted.wild_type)[0,1],2)

In [ ]:
complete_df.loc[complete_df.maxent_score > 3]

In [13]:
total_percentage = complete_df.groupby(['library_key'])['Predicted frequency'].sum().reset_index('library_key')
total_percentage_dict = pd.Series(total_percentage['Predicted frequency'].values, index = total_percentage.library_key).to_dict()

In [14]:
complete_df['Predicted frequency'] = complete_df.apply(lambda x: x['Predicted frequency']/total_percentage_dict[x.library_key], axis=1)

In [15]:
complete_df

,Category,Genotype,Genotype position,Inserted Bases,Length,Predicted frequency,RC,cutsite,library_key,maxent_mer,rc_genotype,maxent_score,maxen_prob,maxent_prob
0,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2.0,NaN,11,0.028148,False,70,9251,CTTGCCACCCATCGCGGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,0.371708,5.918717e-01,5.918717e-01
1,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11.0,NaN,11,0.028148,False,70,9251,ACCCATCGCGGCTCCCCTCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-9.906475,4.984834e-05,4.984834e-05
2,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2.0,NaN,12,0.021922,False,70,9251,ACTTGCCACCCATCGCGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2.179914,8.984312e-01,8.984312e-01
3,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11.0,NaN,12,0.021922,False,70,9251,CACCCATCGCGGCTCCCCTCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-10.393787,3.062122e-05,3.062122e-05
4,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,2.0,NaN,14,0.013296,False,70,9251,CTACTTGCCACCCATCGAAGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,3.041944,9.544334e-01,9.544334e-01
5,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,5.0,NaN,14,0.013296,False,70,9251,CTACTTGCCACCCATCGCGGGGG,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-1.556750,1.741135e-01,1.741135e-01
6,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11.0,NaN,14,0.013296,False,70,9251,GCCACCCATCGCGGCTCCCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-11.634197,8.857857e-06,8.857857e-06
7,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,6.0,NaN,15,0.077555,False,70,9251,CTACTTGCCACCCATCGCGGGGA,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-1.349315,2.059823e-01,2.059823e-01
8,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,11.0,NaN,15,0.010355,False,70,9251,TGCCACCCATCGCGGCTCCCATT,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-11.921106,6.648543e-06,6.648543e-06
9,del,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,13.0,NaN,15,0.009691,False,70,9251,CCACCCATCGCGGCTCCCCTTTA,GTGCACCTGCGGCGCGGGTCACGGGCTTCCCGTGTGGGTGCCTACT...,-13.017140,2.221912e-06,2.221912e-06


In [ ]:
complete_df['probability'] = complete_df.maxent_prob * complete_df['Predicted frequency']

In [ ]:
df = complete_df.groupby('library_key')['score'].sum()

In [ ]:
df = df.reset_index('library_key').rename(columns={'library_key':'id'})

In [ ]:
df.score.sort_values()

In [ ]:
df['wild_type'] = df.id.apply(lambda x: wild_type_dict[x])

Another way of calculating a probability (instead of score): everything above a certain threshold ==> spliced. Weighted average of spliced versus unspliced --> probability for each gRNA/target pair. How to choose threshold? From wildtype?

In [ ]:
plot = plt.scatter(df.score, df.wild_type)
plt.xlabel('predictions (ins and dels)')
plt.ylabel('wild type')
plt.title('Prediction (cas9) vs wildtype score distribution')
round(np.corrcoef(df.score, df.wild_type)[0,1],2)

In [ ]:
score_dict = pd.Series(df.score.values, index = df.id).to_dict()

In [ ]:
wild_dict = pd.Series(df.wild_type.values, index = df.id).to_dict()

In [ ]:
b6 = pd.read_csv('b6_match_all.csv')
b6 = b6.drop('id', axis=1).rename(columns={'real_id':'id'})

In [ ]:
def b_count(b6_match):
    b6_ids = b6_match.id.unique()
    all_scores = b6_match.groupby('id')['slice_type'].value_counts(normalize=True).rename(columns = {'slice_type':'percent'}).reset_index('slice_type').reset_index('id').rename(columns = {0:'percent'})
    b_scores = all_scores.loc[lambda x: x.slice_type == 'b']
    b_score_dict = pd.Series(b_scores.slice_type.values, index = b_scores.id).to_dict()
    for x in b6_ids:
        if x not in b_score_dict:
            b_scores = b_scores.append({'id':x, 'slice_type':'b', 'percent': 0.0}, ignore_index=True)
    return b_scores

In [ ]:
def c_count(b6_match):
    b6_ids = b6_match.id.unique()
    all_scores = b6_match.groupby('id')['slice_type'].value_counts(normalize=True).rename(columns = {'slice_type':'percent'}).reset_index('slice_type').reset_index('id').rename(columns = {0:'percent'})
    c_scores = all_scores.loc[lambda x: x.slice_type == 'c']
    c_score_dict = pd.Series(c_scores.slice_type.values, index = c_scores.id).to_dict()
    for x in b6_ids:
        if x not in c_score_dict:
            c_scores = c_scores.append({'id':x, 'slice_type':'c', 'percent': 0.0}, ignore_index=True)
    return c_scores

In [ ]:
df.loc[df.id == 5]

In [ ]:
c_probability

In [ ]:
c_probability = c_count(b6)
c_probability['score'] = c_probability.id.apply(lambda x: score_dict[x])
# c_probability.score = c_probability.score.apply(lambda x: log_to_prob(x))
plt.scatter(c_probability.percent, c_probability.score)
title = plt.title('probability of c splice (x axis) against maxent score of splice inclusion - wildtype')

## Q: How to convert scores into probability of splicing out?

log  odds frequency of splice inclusion